In [1]:
# --- Imports ---
import pandas as pd

In [2]:
# Load K562 E-Gs
df1 = pd.read_csv('K562/K562.network.tsv', sep = '\t')

print(df1.shape)
df1.head(n = 5)

(78062, 9)


,chrom,chromStart,chromEnd,name,class,targetGene,targetEnsemblID,cellType,MPRabc
0,chr1,9863,10662,intergenic|chr1:9863-10662,intergenic,OR4F5,ENSG00000186092,K562,0.352643
1,chr1,9863,10662,intergenic|chr1:9863-10662,intergenic,FAM138F,ENSG00000282591,K562,0.455790
2,chr1,9863,10662,intergenic|chr1:9863-10662,intergenic,FAM138A,ENSG00000237613,K562,0.455790
3,chr1,29099,29599,intergenic|chr1:29099-29599,intergenic,OR4F5,ENSG00000186092,K562,0.312436
4,chr1,29099,29599,intergenic|chr1:29099-29599,intergenic,FAM138F,ENSG00000282591,K562,0.766666


In [3]:
# Load K562 TFs
df2 = pd.read_csv('K562/K562.FIMO.VERIFIED.tsv', sep = '\t')

df2 = df2.rename(columns = {'chromStart': 'motifStart', 'chromEnd': 'motifEnd'})

print(df2.shape)
df2.head(n = 5)

(1472664, 10)


,chrom,motifStart,motifEnd,name,score,strand,motifID,motifAltID,pValue,matchedSequence
0,chr1,10104,10122,intergenic|chr1:9863-10662,10.79820,+,MA0073.2,RREB1,6.810000e-06,ACCCAACCCTAACCCTAAC
1,chr1,10286,10304,intergenic|chr1:9863-10662,9.94495,+,MA0073.2,RREB1,9.290000e-06,ACCCCAACCCCAACCCCAA
2,chr1,10287,10305,intergenic|chr1:9863-10662,14.08260,+,MA0073.2,RREB1,1.840000e-06,CCCCAACCCCAACCCCAAC
3,chr1,10288,10306,intergenic|chr1:9863-10662,15.75230,+,MA0073.2,RREB1,9.130000e-07,CCCAACCCCAACCCCAACC
4,chr1,10292,10310,intergenic|chr1:9863-10662,9.94495,+,MA0073.2,RREB1,9.290000e-06,ACCCCAACCCCAACCCCAA


In [4]:
# Load K562 eQTLs
df3 = pd.read_csv('eQTLs/Blood.SNV.eQTLs.tsv', sep = '\t')

df3 = df3.rename(columns = {'chromStart': 'varStart', 'chromEnd': 'varEnd'})

print(df3.shape)
df3.head(n = 5)

(2486143, 8)


,chrom,varStart,varEnd,ref,alt,targetGene,targetEnsemblID,pValue
0,chr1,64763,64764,C,T,WASH7P,ENSG00000227232,1.268920e-07
1,chr1,135202,135203,G,A,WASH7P,ENSG00000227232,1.268920e-07
2,chr1,625775,625776,T,G,WASH7P,ENSG00000227232,1.268920e-07
3,chr1,634552,634553,G,A,WASH7P,ENSG00000227232,1.268920e-07
4,chr1,665097,665098,G,A,WASH7P,ENSG00000227232,1.268920e-07


In [ ]:
import pandas as pd

motif_regions = (
    df2
    .merge(
        df1[['chrom', 'chromStart', 'chromEnd', 'name', 'class',
             'targetGene', 'targetEnsemblID', 'MPRabc']],
        on=['chrom', 'name'],
        how='inner'
    )
)

print("Motifs assigned to enhancers / genes:", motif_regions.shape)

motif_eqtl_pairs = (
    motif_regions
    .merge(
        df3,                      # Blood eQTLs
        on=['chrom', 'targetGene'],
        how='inner',
        suffixes=('', '_eQTL')
    )
)

print("Motif–enhancer–eQTL triples before interval filter:", motif_eqtl_pairs.shape)

perturbing_eqtls = motif_eqtl_pairs[
    (motif_eqtl_pairs['varStart'] < motif_eqtl_pairs['motifEnd']) &
    (motif_eqtl_pairs['varEnd']   > motif_eqtl_pairs['motifStart'])
].copy()

print("eQTLs that perturb motifs (and match targetGene):", perturbing_eqtls.shape)
perturbing_eqtls.head()

Motifs assigned to enhancers / genes: (2266108, 16)


In [ ]:
perturbing_eqtls[perturbing_eqtls['motifAltID'] == 'GATA1::TAL1']